In [1]:
import numpy as np
import pandas as pd
import sys
from camel_converter import to_snake

In [2]:
def remove_null_values(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df.columns[df.isna().mean() * 100 < 60]]
    df = df[df.isna().mean(axis=1) * 100 < 50]
    return df

In [3]:
df_18 = remove_null_values(pd.read_csv("/home/adnan/Documents/codes/data_Science/eda-and-visualization/IT-salary/data/raw/IT Salary Survey EU 2018.csv"))
df_19 = remove_null_values(pd.read_csv("/home/adnan/Documents/codes/data_Science/eda-and-visualization/IT-salary/data/raw/IT Salary Survey EU 2019.csv"))
df_20 = remove_null_values(pd.read_csv("/home/adnan/Documents/codes/data_Science/eda-and-visualization/IT-salary/data/raw/IT Salary Survey EU  2020.csv"))

In [4]:
df_20 = df_20.drop(columns=['Have you lost your job due to the coronavirus outbreak?', 
            'Other technologies/programming languages you use often', 'Employment status', 
            'Yearly bonus + stocks in EUR', 'Years of experience in Germany',
            'Annual bonus+stocks one year ago. Only answer if staying in same country'])

df_19 = df_19.drop(columns=['Company business sector', 'Yearly bonus', 'Number of home office days per month'])

df_18 = df_18.drop(columns=["Are you getting any Stock Options?", "Salary two years ago"])


In [5]:
df_20 = df_20.rename(columns={
    'Total years of experience' : 'experience',
    'Your main technology / programming language' : 'programming_language',
    'Yearly brutto salary (without bonus and stocks) in EUR' : 'salary',
    'Annual brutto salary (without bonus and stocks) one year ago. Only answer if staying in the same country' : \
    'Salary one year ago',
    'Number of vacation days' : 'vacation_days',
    'Main language at work' : 'language',
    'Position ' : 'Position'})

In [6]:
df_19 = df_19.rename(columns={
    'Zeitstempel' : 'Timestamp',
    'Position (without seniority)' : 'Position',
    'Years of experience' : 'experience',
    'Your main technology / programming language' : 'programming_language',
    'Yearly brutto salary (without bonus and stocks)' : 'salary',
    'Yearly brutto salary (without bonus and stocks) one year ago. Only answer if staying in same country' : \
    'Salary one year ago',
    'Number of vacation days' : 'vacation_days',
    'Main language at work' : 'language'
})

In [7]:
df_18 = df_18.rename(columns={
    'Main language at work' : 'language',
    'Years of experience' : 'experience',
    'Your level' : 'Seniority level',
    'Current Salary' : 'salary'
})

In [8]:
df_20['survey_year'] = 2020
df_19['survey_year'] = 2019
df_18['survey_year'] = 2018

In [9]:
def func(value: str, array: list[str | int]) -> str:
    if value is np.nan:
        return np.nan
    value_list = str(value).lower().split(" ")
    result_list = []
    for each in value_list:
        if each not in array:
            out = each.replace("(", " ").replace(")", " ").strip()
            result_list.append(out)
    if result_list == []:
        return np.nan
    return " ".join(result_list)
positions = ["software", "engineer", 'designer', 'developer', "developer", 'qa', 'analyist', 'senior', \
            'project', 'manager', 'team', 'lead']
df_18['programming_language'] = df_18['Position'].apply(lambda x: func(x, positions))

In [10]:
df_20.columns = ["_".join(to_snake(each).split(" ")) for each in df_20.columns]
df_19.columns = ["_".join(to_snake(each).split(" ")) for each in df_19.columns]
df_18.columns = ["_".join(to_snake(each).split(" ")) for each in df_18.columns]

In [11]:
merged_df = pd.concat([df_18, df_19, df_20], axis=0)

In [12]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2989 entries, 0 to 1252
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   timestamp             2989 non-null   object 
 1   age                   2778 non-null   float64
 2   gender                2980 non-null   object 
 3   city                  2971 non-null   object 
 4   position              2972 non-null   object 
 5   experience            2958 non-null   object 
 6   seniority_level       2958 non-null   object 
 7   salary                2986 non-null   float64
 8   salary_one_year_ago   2084 non-null   float64
 9   language              2973 non-null   object 
 10  company_size          2962 non-null   object 
 11  company_type          2917 non-null   object 
 12  survey_year           2989 non-null   int64  
 13  programming_language  2705 non-null   object 
 14  vacation_days         2116 non-null   object 
 15  сontract_duration     2186

In [13]:
from collections import Counter

columns = [*df_19.columns, *df_18.columns, *df_20.columns]
Counter(columns)

Counter({'timestamp': 3,
         'age': 3,
         'gender': 3,
         'city': 3,
         'seniority_level': 3,
         'position': 3,
         'experience': 3,
         'programming_language': 3,
         'salary': 3,
         'salary_one_year_ago': 3,
         'language': 3,
         'company_size': 3,
         'company_type': 3,
         'survey_year': 3,
         'vacation_days': 2,
         'сontract_duration': 2})

In [14]:
merged_df.to_csv("/home/adnan/Documents/codes/data_Science/eda-and-visualization/IT-salary/data/processed/initial_cleaned_IT_survey.csv", index=False)